# Carga de datos

In [2]:
import pymongo
import folium

In [5]:
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Limpieza

# Exploración

# Gráficas

### Plot 1

### Plot 2

### Plot 3

### Plot 4

### Plot 5